In [1]:
import numpy as np
import scipy.io
import pandas
from tools.countries import*
from scipy import linalg
import time

In [2]:
#works!
BK, Time, stage1, Max, Mean, Test_time  = GSSA_country_showcase()

A cache file so we dont need to run another 2 to 3 hours:

In [5]:
import pandas as pd

cache = pd.DataFrame(
    {'Max': Max,
     'Mean': Mean,
     'Test_Time': Test_time,
     "stage1": stage,
     "BK":BK
    })

In [8]:
cache.to_csv("Country_showcase.csv", index=None)

In [9]:
test = pd.read_csv("Country_showcase.csv")

Simulation part:

In [65]:
#####User defined parameters
N = 2
T = 2000
#####Model parameters
gam = 1
alpha = 0.36
beta = 0.99
delta = 0.025
rho = 0.95
sigma = 0.01

######################################
####Variance-covariance matrix
vcv = sigma**(2)*(np.eye(N)+ np.ones(N))
####Normalising constant
A = (1-beta+beta*delta)/alpha/beta
####welfare weight
tau = 1
######################################
#The above normalization ensures that steady state
k = np.ones((T+1,N))
a = np.ones((1,N))
#####################################
#construct productivity levels
a20200 = Productivity(T,N,a,sigma,rho)

GSSA parameters

In [4]:
#checking part with matlab
a20200 = scipy.io.loadmat("aT20200N10.mat").get("a20200")
a20200 = a20200[:T, :N]

In [4]:
#Compute a first-degree polynomial solution using the one-node Monte Carlo integration method (this #solution will be used as an initial guess for the other cases) 

start = time.time()
kdamp = 0.1
dif_1d = 1e+10
# Initialize the first-degree capital policy functions of N countries 
bk_1d= np.vstack((np.zeros((1,N)), np.diag(0.9*np.ones(N)),np.diag(0.1*np.ones(N))))
# Initialize the capital series
k_old = np.ones((T+1,N))
# The main iterative cycle of GSSA
while dif_1d > 1e-4*kdamp:
    #construct starting numpy for value storages
    x = np.empty((T,2*N+1))
    for i in range(T):
        x[i]= np.hstack((1,k[i], a20200[i]))
        k[i+1]= np.hstack((1,k[i], a20200[i]))@bk_1d
        #Compute consumption series 
    C = (A*k[:T]**alpha*a20200[:T]-k[1:T+1]+(1-delta)*k[:T])@np.ones((N,1))
    #Individual consumption is the same for all countries
    c = C@np.ones((1,N))/N

    #Evaluate the percentage (unit-free) difference between the series from the previous and current iterations
    dif_1d = np.mean(abs(1-k/k_old))
    #Monte Carlo realizations of the right side of the Euler equation
    #Consturct containers
    Y = np.empty((T-1,N))
    for i in range(N):
        Y[:T-1,i]= beta*c[1:T,i]**(-gam)/c[:T-1,i]**(-gam)*(1-delta+alpha*A*k[1:T,i]**(alpha-1)*a20200[1:T,i])*k[1:T,i]

    #Compute and update the coefficients of the capital policy functions
    bk_hat_1d = linalg.inv(x[:T-1].conj().T@x[:T-1])@x[:T-1].conj().T@Y[:T-1]
    bk_1d = kdamp*bk_hat_1d+(1-kdamp)*bk_1d
    k_old = k.copy()
end = time.time()
Ncountry_GSSA_1stage = end-start

Stage 2:

In [5]:
#Compute polynomial solutions of the degrees from one to D_max using one of the following integration methods: Monte Carlo, Gauss-Hermite product and monomial non-product methods


#Damping parameter for (fixed-point) iteration on the coefficients of the capital policy functions
kdamp     = 0.1
#Set the initial difference between the series from two iterations in the convergence criterion   
dif_GSSA_D  = 1e+10

#The matrix of the polynomial coefficients
D_max = 5

#construct container
npol = np.empty((1,D_max))

for i in range(1, D_max+1):
    npol[0,i-1]= Ord_Polynomial_N(np.hstack((k[0],a20200[0]))[np.newaxis,], i).shape[1]

#Choose an integration method here
###
IM = 11

##################################
if ((IM>=1) and (IM<=10)):
    n_nodes, epsi_nodes, weight_nodes = GH_Quadrature(IM, N, vcv)
elif IM == 11:
    n_nodes, epsi_nodes, weight_nodes = Monomials_1(N,vcv)
elif IM == 12:
    n_nodes, epsi_nodes, weight_nodes = Monomials_2(N,vcv)

#14. Choose a regression method and specifications
#####only 6 methods avaliable here !!!!####

RM    = 5       
normalize = 1
penalty = 7 

#Compute the polynomial solutions of the degrees from one to D_max

#Construct empty container for results
BK = []
Time = []

In [10]:
D = 2
start = time.time()
X = Ord_Polynomial_N(np.hstack((k[:T,:],a20200[:T,:])),D)
bk_D = Num_Stab_Approx(X[:T-1,:],Y[:T-1,:], RM, penalty, normalize)
k_old = np.ones((T+1,N))
dif_GSSA_D  = 1e+10
while dif_GSSA_D > 1e-4/10**D*kdamp:
    #Generate time series of capital
    for i in range(1, T+1):
        X[i-1,] = Ord_Polynomial_N(np.hstack((k[i-1,:], a20200[i-1, :]))[np.newaxis,], D)
        k[i,] = X[i-1,][np.newaxis,]@bk_D

    #15.2.2Compute consumption series of all countries:
    ###################################################

    # N current capital stocks  
    k0 = k[0:T,]
    # N current productivity levels 
    a0 = a20200[0:T,]
    # N next-period capital stocks
    k1 = k[1:T+1,]
    # Aggregate consumption is computed by summing up individual consumption, which in turn, is found from the individual budget constraints
    C = (A*k0**alpha*a0 - k1+ (1-delta)*k0)@np.ones((N,1))

    # Individual consumption is the same for all countries, check JMM (2011)
    c = C@np.ones((1,N))/N

    #15.2.3 Approximate the conditional expectations for t=1,...T-1 using the integration method chosen

    #15.2.3.1 The one-node Monte Carlo integration method approximates the 
    #values of the conditional expectations, Y, in the Euler equation with
    # the realization of the integrand in the next period
    if IM == 0:
        for i in range(1, N+1):
            Y[0:T-1,i-1][:,np.newaxis] = beta*c[1:T,i-1][:,np.newaxis]**(-gam)/c[0:T-1, i-1][:,np.newaxis]**(-gam)*(1-delta+alpha*A*k[1:T,i-1][:,np.newaxis]**(alpha-1)*a20200[1:T,i-1][:,np.newaxis])*k[1:T,i-1][:,np.newaxis]

    #15.2.3.2 Deterministic integration methods approximate the values of 
    # conditional expectations, Y, in the Euler equation as a weighted average 
    # of the values of the integrand in the given nodes with the given weights 
    else:
        Y = np.zeros((T,N))
        for i in range(1, n_nodes+1):
            # Compute the next-period productivity levels for each integration node using condition (C3) 
            # in JMM (2011)
            a1 = a20200[0:T,:]**rho*np.exp(np.ones((T,1))@epsi_nodes[i-1,:][np.newaxis,])

            # Compute capital of period t+2 (chosen at t+1) using the capital policy functions
            k2 = Ord_Polynomial_N(np.hstack((k1, a1)),D)@bk_D

            # C is computed by summing up individual consumption, which in turn, is from the individual budget constraints
            C1 = (A*k1**alpha*a1-k2+(1-delta)*k1)@np.ones((N,1))

            # Compute next-period individual consumption for N countries 
            c1 = C1@np.ones((1,N))/N
            for i in range(1, N+1):
                Y[0:T,i-1][:,np.newaxis]= Y[0:T,i-1][:,np.newaxis]+ weight_nodes[i-1, 0]*beta*c1[0:T, i-1][:,np.newaxis]**(-gam)/c[0:T,i-1][:,np.newaxis]**(-gam)*(1-delta+alpha*A*k1[0:T,i-1][:,np.newaxis]**(alpha-1)*a1[0:T,i-1][:,np.newaxis])*k1[0:T, i-1][:,np.newaxis]

    # 15.2.4 Evaluate the percentage (unit-free) difference between the 
    # capital series from the previous and current iterations
    #-------------------------------------------------------
    dif_GSSA_D = np.mean(abs(1-k/k_old))

    # 15.2.5 Compute and update the coefficients of the capital policy 
    # functions 
    # ----------------------------------------------------------------
    # Compute new coefficients of the capital 
    # policy functions using the chosen 
    # approximation method
    bk_hat_D = Num_Stab_Approx(X[0:T-1,:], Y[0:T-1,:], RM, penalty, normalize)
    bk_D = kdamp*bk_hat_D+ (1-kdamp)*bk_D

    #15.2.6 Store the capital series 
    #--------------------------------
    k_old = k.copy()
end = time.time()
BK.append(bk_D)
Time.append(end-start)

Accuracy Test

In [214]:
# 16. Accuracy test of the GSSA solutions: errors on a stochastic simulation
# 16.1 Specify a set of points on which the accuracy is evaluated

#predetermined values from GSSA
a20200 = scipy.io.loadmat("aT20200N10.mat").get("a20200")
N = 2
T = 2000
D = 1
j = 1
IM = 11
gam = 1
alpha = 0.36
beta = 0.99
delta = 0.025
rho = 0.95
sigma = 0.01
D_max = 5
vcv = sigma**(2)*(np.eye(N)+ np.ones(N))
#Choose the simulation length for the test on a stochastic simulation, T_test<=10,200
T_test = 10200

#Restrict the series of the productivity levels for the test on a stochastics
a_test = a20200[T:T+T_test,0:N]

#16.2 Choose an integration method for evaluating accuracy of solutions
IM_test = 11

#16.3 Compute errors on a stochastic simulation for the GSSA polynomial solution of degrees D=1,...,D_max
Max = []
Mean = []
Test_time = []
for D in range(1, D_max+1):
    #16.3.1 Simulate the time series solution under the given capital-
    # policy-function coefficients, BK(:,:,D) with D=1,...,D_max 
    bk = BK[D-1]
    #Initial condition for capital (equal to steady state)
    k_test = np.ones((1,N))

    for j in range(1, T_test):
        X_test = Ord_Polynomial_N(np.hstack((k_test[j-1][np.newaxis,],a_test[j-1,][np.newaxis,])), D)
        k_test = np.vstack((k_test, X_test@bk))
    
    #16.3.2 Errors across 10,000 points on a stochastic simulation
    discard = 200
    Errors_mean, Errors_max, time_test = Accuracy_Test_N(k_test, a_test, bk, D, IM_test, alpha, gam, delta, beta, A, tau, rho, vcv, discard)
    Max.append(Errors_max)
    Mean.append(Errors_mean)
    Test_time.append(time_test)

Accuracy_countries_n

In [60]:
#change a to a
#change k to k

#Delete above when functionising
P, N = a.shape

#2. Integration method for evaluating accuracy 
if ((IM>=1) and (IM<=10)):
    n_nodes, epsi_nodes, weight_nodes = GH_Quadrature(IM, N, vcv)
elif IM == 11:
    n_nodes, epsi_nodes, weight_nodes = Monomials_1(N,vcv)
elif IM == 12:
    n_nodes, epsi_nodes, weight_nodes = Monomials_2(N,vcv)

# 3. Polynomial bases for the test
X = Ord_Polynomial_N(np.hstack((k,a)),D)

# 4. Given the solution for capital, compute consumption on the given set of points
Error =[]
for i in range(1, P+1):
    # 4.1 Variables in point p
    i = 1
    # N capital stocks of period t
    k0 = k[i-1,:N][np.newaxis,:]
    # N productivity levels of period t
    a0 = a[i-1,:N][np.newaxis,:]
    # Complete (second-degree) polynomial
    X0 = X[i-1, :][np.newaxis,:]

    #4.2 Capital and consumption choices at t

    # Compute a row-vector of capital of period t+1 (chosen at t) using
    # the corresponding capital policy functions
    k1 = X0@bk

    C0 = (A*k0**alpha*a0 - k1 + (1-delta)*k0)@np.ones((N,1))

    c0 = C0@np.ones((1,N))/N
    #4.3 Capital and consumption choices at t+1

    #Compute the next-period productivity levels in each integration node
    a1 = (np.ones((n_nodes,1))@a0)**rho*np.exp(epsi_nodes)

    #Duplicate k1 n_nodes times to create a matrix with n_nodes identical rows
    k1_dupl = np.ones((n_nodes,1))@k1

    # Form a complete polynomial of degree D (at t+1) in the given point 
    X1 = Ord_Polynomial_N(np.hstack((k1_dupl,a1)),D)

    #Compute capital of period t+2 (chosen at t+1) using the second-degree capital policy functions
    k2 = X1@bk

    # Aggregate consumption is computed by summing up individual consumption, which in turn, is found from the individual budget constraints
    C1 = (A*k1_dupl**alpha*a1-k2+ (1-delta)*k1_dupl)@np.ones((N,1))

    c1 = C1@np.ones((1,N))/N

    #5. Approximation errors in point p
    # 5.1 Lagrange multiplier associated with the aggregate resource constraint
    #Compute a country's marginal utility of consumption multiplied by its welfare weight
    MUC0j = np.empty((1,N))
    for j in range(1,N+1):
        MUC0j[0,j-1] = tau*c0[0, j-1]**(-gam)
    # An optimality condition w.r.t. consumption of period t equates 
    # the Lagrange multiplier of the aggregate resource constraint of 
    # period t and each country's marginal utility of consumption 
    # multiplied by its welfare weight; to infer the Lagrange multiplier,  
    # we average across N countries
    lambda0 = np.mean(MUC0j, axis=1)[np.newaxis,:]

    MUC1j = np.empty((n_nodes, N))

    #Compute a country's marginal utility of consumption multiplied by its welfare weight
    for j in range(1,N+1):
        MUC1j[:n_nodes, j-1] = tau*c1[:n_nodes,j-1]**(-gam)
    # Similarly, the Lagrange multiplier of the aggregate resource 
    # constraint of period t+1 is equal to a country's marginal utility 
    # of consumption multiplied by its welfare weight; to infer the 
    # Lagrange multiplier, we average across N countries
    lambda1 = np.mean(MUC1j, axis=1)[:,np.newaxis]

    #5.2 Unit-free Euler-equation errors
    Errors1 = np.zeros((1,N))
    for j in range(1, N+1):
        Errors1[0,j-1] = 1- weight_nodes.conj().T@(beta*lambda1/lambda0*(1-delta+alpha*A*k1[0,j-1]**(alpha-1)*a1[:n_nodes,j-1])[:,np.newaxis])
    # 5.2 Unit-free errors in the optimality conditions w.r.t. consumption
    Errors2 = np.zeros((1,N))
    for j in range(1, N+1):
        Errors2[0,j-1] = 1- lambda0/(tau*c0[0,j-1]**(-gam))
    # 5.3 Unit-free errors in the optimality conditions w.r.t. labor 
    Errors3 = np.zeros((1, N))
    # 5.4 Unit-free approximation error in the aggregate resource constraint
    Errors4 = 1- (c0[0,:N][np.newaxis,:] + k1[0,0:N][np.newaxis,:]-(1-delta)*k0[0,0:N])@np.ones((N,1))/(A*k0[0,0:N][np.newaxis,:]**alpha*a0[0,0:N][np.newaxis,:]@np.ones((N,1)))
    # 5.5 Approximation errors in the capital-accumulation equation
    Errors5 = np.zeros((1,N))

    # combine it into a row
    # First we flatten it in order to store the values in a list
    Error_final = np.hstack((Errors1, Errors2, Errors3, Errors4, Errors5)).flatten()
    Error.append(Error_final)
#Restore it to 2D
Error = np.stack(Error, axis=0)

# 6.1 Approximation errors across all the optimality conditions
# Average absolute approximation error 
Errors_mean = math.log10(np.mean(np.abs(Error[discard:,])))
# Maximum absolute approximation error
Error_max = math.log10(np.max(np.abs(Error[discard:])))
# 6.2 is not touched as it is not relevent for the results

Num_Stab_Approx(X,Y,RM,penalty,normalize)

In [7]:
X = X[:T-1,:]
Y = Y[:T-1,:]
T, n = X.shape
N  = Y.shape[1]
X1 = (X[:,1:n]-np.ones((T,1))@X[:,1:n].mean(axis=0)[np.newaxis,])/(np.ones((T,1))@np.std(X[:,1:n], axis=0, ddof=1)[np.newaxis,])
Y1 = (Y - np.ones((T,1))@Y.mean(axis=0)[np.newaxis,])/(np.ones((T,1))@np.std(Y, axis=0, ddof= 1)[np.newaxis,])
n1 = n-1



In [15]:
U, S, Vh = linalg.svd(X1, full_matrices=False)
V = Vh.T
r = np.count_nonzero(np.divide(np.diag(S).max(), np.diag(S))<= 10**(penalty))

In [10]:
r = 3

In [14]:
np.diag(np.divide(1., S))

array([[0.0159787 , 0.        , 0.        ],
       [0.        , 0.01911957, 0.        ],
       [0.        , 0.        , 0.02794562]])

In [ ]:
for i in 